### 1. Calcular la duración media del alquiler (en días) para cada película:

```sql
SELECT
	DISTINCT f.title,
	f.rental_duration,
	AVG(COALESCE(DATEDIFF(r.return_date, r.rental_date), 0)) AS 'avg_rental_duration'
FROM
	film f
INNER JOIN inventory i ON
	i.film_id = f.film_id
INNER JOIN rental r ON
	r.inventory_id = i.inventory_id
GROUP BY
	f.film_id;
```

|title|rental_duration|avg_rental_duration|
|-----|---------------|-------------------|
|ACADEMY DINOSAUR|6              |4.9850             |
|ACE GOLDFINGER|3              |4.9850             |
|ADAPTATION HOLES|7              |4.9850             |
|AFFAIR PREJUDICE|5              |4.9850             |
|AFRICAN EGG|6              |4.9850             |
|AGENT TRUMAN|3              |4.9850             |
|AIRPLANE SIERRA|6              |4.9850             |
|AIRPORT POLLOCK|6              |4.9850             |
|ALABAMA DEVIL|3              |4.9850             |
|ALADDIN CALENDAR|6              |4.9850             |
|ALAMO VIDEOTAPE|6              |4.9850             |
|ALASKA PHANTOM|6              |4.9850             |
|ALI FOREVER|4              |4.9850             |
|ALICE FANTASIA|6              |4.9850             |


### 2. Calcular el importe medio de los pagos para cada miembro del personal:

```sql
SELECT s.staff_id, AVG(p.amount) AS 'avg_payment_amount'
FROM staff s
INNER JOIN rental r ON r.staff_id = s.staff_id
INNER JOIN payment p ON p.rental_id = r.rental_id
GROUP BY s.staff_id;
```

#### Output

|staff_id|avg_payment_amount|
|--------|------------------|
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |

### 3. Calcular los ingresos totales para cada cliente, mostrando el total acumulado dentro del historial de alquileres de cada cliente:

```sql
SELECT
	c.customer_id,
	r.rental_id,
	r.rental_date,
	p.amount,
	SUM(p.amount) OVER (
	ORDER BY c.customer_id,
	r.rental_id) AS 'running_total'
FROM
	customer c
INNER JOIN rental r ON
	r.customer_id = c.customer_id
INNER JOIN payment p ON
	p.rental_id = r.rental_id;
```

#### Output

|customer_id|rental_id|rental_date        |amount|running_total|
|-----------|---------|-------------------|------|-------------|
|1          |76       |2005-05-25 11:30:37|2.99  |2.99         |
|1          |573      |2005-05-28 10:35:23|0.99  |3.98         |
|1          |1185     |2005-06-15 00:54:12|5.99  |9.97         |
|1          |1422     |2005-06-15 18:02:53|0.99  |10.96        |
|1          |1476     |2005-06-15 21:08:46|9.99  |20.95        |
|1          |1725     |2005-06-16 15:18:57|4.99  |25.94        |
|1          |2308     |2005-06-18 08:41:48|4.99  |30.93        |
|1          |2363     |2005-06-18 13:33:59|0.99  |31.92        |
|1          |3284     |2005-06-21 06:24:45|3.99  |35.91        |
|1          |4526     |2005-07-08 03:17:05|5.99  |41.90        |

### 4. Determinar el cuartil para las tarifas de alquiler de las películas

```sql
SELECT
	f.title,
	f.rental_rate,
	NTILE(4) OVER (
	ORDER BY f.rental_rate) AS 'quartile'
FROM
	film f
ORDER BY
	f.rental_rate,
	f.title;
```

#### Output

|title|rental_rate|quartile           |
|-----|-----------|-------------------|
|ACADEMY DINOSAUR|0.99       |1                  |
|ALAMO VIDEOTAPE|0.99       |1                  |
|ALASKA PHANTOM|0.99       |1                  |
|ALICE FANTASIA|0.99       |1                  |
|ALONE TRIP|0.99       |1                  |
|ALTER VICTORY|0.99       |1                  |
|AMADEUS HOLY|0.99       |1                  |
|ANACONDA CONFESSIONS|0.99       |1                  |
|ANNIE IDENTITY|0.99       |1                  |
|ANONYMOUS HUMAN|0.99       |1                  |


### 5. Determinar la primera y última fecha de alquiler para cada cliente:

```sql
SELECT c.customer_id, MIN(r.rental_date) AS 'first_rental_date', MAX(r.rental_date) AS 'last_rental_date'
FROM customer c
LEFT JOIN rental r ON r.customer_id = c.customer_id
GROUP BY c.customer_id;
```

#### Expected result

|customer_id|first_rental_date|last_rental_date   |
|-----------|-----------------|-------------------|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|


### 6. Calcular el rango de los clientes basado en el número de sus alquileres:

```sql
SELECT
	r.customer_id,
	COUNT(*) AS 'rental_count',
	RANK() OVER (ORDER BY COUNT(*) DESC) AS 'rental_count_rank'
FROM
	rental r
GROUP BY
	r.customer_id;
```

#### Expected output

|customer_id|rental_count                 |rental_count_rank|
|-----------|-----------------------------|-----------------|
|148        |46                           |1                |
|526        |45                           |2                |
|144        |42                           |3                |
|236        |42                           |3                |
|75         |41                           |5                |
|197        |40                           |6                |
...|...|...

### 7. Calcular el total acumulado de ingresos por día para la categoría de películas 'Familiar':

```sql
SELECT
	DISTINCT f.title,
	r.rental_date,
	p.amount,
	rev.amount AS 'daily_revenue'
FROM
	category c
INNER JOIN film_category fc ON
	fc.category_id = c.category_id
INNER JOIN film f ON
	f.film_id = fc.film_id
INNER JOIN inventory i ON
	i.film_id = f.film_id
INNER JOIN rental r ON
	r.inventory_id = i.inventory_id
INNER JOIN payment p ON
	p.rental_id = r.rental_id
LEFT JOIN (
	SELECT
		CAST(r2.rental_date AS DATE) AS 'date',
		i2.film_id,
		SUM(p2.amount) AS 'amount'
	FROM
		rental r2
	INNER JOIN payment p2 ON
		p2.rental_id = r2.rental_id
	INNER JOIN inventory i2 ON
		i2.inventory_id = r2.inventory_id
	GROUP BY
		1,
		i2.film_id
	) rev ON rev.film_id = f.film_id AND rev.`date` = CAST(r.rental_date AS DATE)
WHERE
	c.name = 'Family'
ORDER BY
	2 ASC;
```

#### Expected ouput

|film_category|rental_date                  |amount|daily_revenue|
|-------------|-----------------------------|------|-------------|
|BLANKET BEVERLY|2005-05-24 22:53:30          |2.99  |2.99         |
|GRADUATE LORD|2005-05-24 23:03:39          |3.99  |3.99         |
|LAWLESS VISION|2005-05-24 23:31:46          |4.99  |4.99         |
|HANGING DEEP |2005-05-25 00:02:21          |5.99  |5.99         |
|MONTEREY LABYRINTH|2005-05-25 00:31:15          |0.99  |0.99         |
|SCISSORHANDS SLUMS|2005-05-25 01:48:41          |2.99  |2.99         |
|EASY GLADIATOR|2005-05-25 03:41:50          |4.99  |4.99         |

### 8. Asignar un ID único a cada pago dentro del historial de pagos de cada cliente:

```sql
SELECT
	customer_id,
	payment_id,
	ROW_NUMBER() OVER (PARTITION BY customer_id) AS 'payment_sequence_id'
FROM
	payment
ORDER BY
	customer_id,
	payment_date;
```

|customer_id|payment_id|payment_sequence_id|
|-----------|----------|-------------------|
|1          |1         |1                  |
|1          |2         |2                  |
|1          |3         |3                  |
|1          |4         |4                  |
|1          |5         |5                  |
|1          |6         |6                  |
|1          |7         |7                  |
|1          |8         |8                  |
|1          |9         |9                  |
|1          |10        |10                 |
|1          |11        |11                 |
|1          |12        |12                 |
|1          |13        |13                 |
|1          |14        |14                 |
|1          |15        |15                 |
|1          |16        |16                 |
|1          |17        |17                 |
|1          |18        |18                 |
|1          |19        |19                 |
|1          |20        |20                 |
|1          |21        |21                 |
|1          |22        |22                 |
|1          |23        |23                 |
|1          |24        |24                 |
|1          |25        |25                 |
|1          |26        |26                 |
|1          |27        |27                 |
|1          |28        |28                 |
|1          |29        |29                 |
|1          |30        |30                 |
|1          |31        |31                 |
|1          |32        |32                 |
|2          |33        |1                  |
|2          |34        |2                  |
|2          |35        |3                  |
|2          |36        |4                  |
|2          |37        |5                  |
|2          |38        |6                  |
|2          |39        |7                  |
|2          |40        |8                  |
|2          |41        |9                  |
|2          |42        |10                 |
|2          |43        |11                 |
|2          |44        |12                 |


### 9. Calcular la diferencia en días entre cada alquiler y el alquiler anterior para cada cliente:

```sql
SELECT
	c.customer_id,
	r.rental_id,
	r.rental_date,
	LAG(r.rental_date) OVER (PARTITION BY r.customer_id
ORDER BY
	r.rental_date) AS 'previous_rental_date',
	DATEDIFF(r.rental_date, LAG(r.rental_date) OVER (PARTITION BY r.customer_id
ORDER BY
	r.rental_date)) AS 'days_between_rentals'
FROM
	customer c
INNER JOIN rental r ON
	r.customer_id = c.customer_id;
```

#### Expected output


|customer_id|rental_id|rental_date        |previous_rental_date|days_between_rentals|
|-----------|---------|-------------------|--------------------|--------------------|
|1          |76       |2005-05-25 11:30:37|NULL                |NULL                |
|1          |573      |2005-05-28 10:35:23|2005-05-25 11:30:37 |3                   |
|1          |1185     |2005-06-15 00:54:12|2005-05-28 10:35:23 |18                  |
|1          |1422     |2005-06-15 18:02:53|2005-06-15 00:54:12 |0                   |
|1          |1476     |2005-06-15 21:08:46|2005-06-15 18:02:53 |0                   |
|1          |1725     |2005-06-16 15:18:57|2005-06-15 21:08:46 |1                   |
|1          |2308     |2005-06-18 08:41:48|2005-06-16 15:18:57 |2                   |
|1          |2363     |2005-06-18 13:33:59|2005-06-18 08:41:48 |0                   |
|1          |3284     |2005-06-21 06:24:45|2005-06-18 13:33:59 |3                   |
|1          |4526     |2005-07-08 03:17:05|2005-06-21 06:24:45 |17                  |
|1          |4611     |2005-07-08 07:33:56|2005-07-08 03:17:05 |0                   |

